In [15]:
import os
import pickle
import matplotlib.pyplot as plt
import networkx as nx
import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


In [2]:
if not os.path.exists("apkdump"):
    os.mkdir("apkdump")
    

In [19]:
Graph_data = [] #list of tuples (G, lable)
for i in os.listdir("apkdump"):
    print(f'processing : {i}')
    gdata = open("apkdump/"+str(i),'rb')
    db = pickle.load(gdata)
    l = db['label'][0]
    adj = db['adjacency']
    xg = nx.from_dict_of_lists(adj)
    g = dgl.DGLGraph(xg)
    Graph_data.append((g,l))
    
    
    
    
    

processing : agency.contra.apps.kpmg.kpmgglobaltaxevent.apk_networkData.pickle
processing : age.of.civilizations.jakowski.lite.apk_networkData.pickle
processing : agagswe.technological.news.apk_networkData.pickle
processing : air.AdditionIsFun.apk_networkData.pickle
processing : air.air.com.jessoft.flvplayer.FLVPlayer.apk_networkData.pickle
processing : air.air.FootballTacticsAndroid.apk_networkData.pickle
processing : air.air.com.hi4u4o.game.Turbo_Boat_Race.apk_networkData.pickle
processing : aero.sita.lab.resmobileweb.android.mh.apk_networkData.pickle
processing : advanced.speed.booster.apk_networkData.pickle
processing : air.AfterInjuryFrozenAnnaAPK.apk_networkData.pickle


In [20]:
Graph_data

[(DGLGraph(num_nodes=6464, num_edges=33617,
           ndata_schemes={}
           edata_schemes={}), 0), (DGLGraph(num_nodes=27015, num_edges=165471,
           ndata_schemes={}
           edata_schemes={}), 0), (DGLGraph(num_nodes=47823, num_edges=294135,
           ndata_schemes={}
           edata_schemes={}), 0), (DGLGraph(num_nodes=41470, num_edges=247958,
           ndata_schemes={}
           edata_schemes={}), 0), (DGLGraph(num_nodes=8570, num_edges=50530,
           ndata_schemes={}
           edata_schemes={}), 0), (DGLGraph(num_nodes=226, num_edges=1168,
           ndata_schemes={}
           edata_schemes={}), 0), (DGLGraph(num_nodes=4407, num_edges=27762,
           ndata_schemes={}
           edata_schemes={}), 0), (DGLGraph(num_nodes=38932, num_edges=248484,
           ndata_schemes={}
           edata_schemes={}), 0), (DGLGraph(num_nodes=32442, num_edges=192284,
           ndata_schemes={}
           edata_schemes={}), 0), (DGLGraph(num_nodes=4588, num_edges=24993,
   

In [21]:
def collate(samples):
  graphs, labels = map(list, zip(*samples))
  # print(len(samples[0]))
  batched_graph = dgl.batch(graphs)
  return batched_graph, torch.tensor(labels)

In [25]:
#send a message of node feature h.
msg = fn.copy_src(src='h',out='m')
# fn.copy_u()
def reduce(nodes):
    
    
    ''' Avg of the node features'''
    accum = torch.mean(nodes.mailbox['m'],1)
    return {'h': accum}

class NodeApplyModule(nn.Module):
    ''' hv - > ReLU(Whv+b)'''
    def __init__(self, in_feats, out_feats, activation):
        
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_features=in_feats, out_features=out_feats)
        self.activation = activation
    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}
class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN,self).__init__()
        self.apply_mod = NodeApplyModule(in_feats, out_feats, activation)
def forward(self, g, features):
    #initialize node feature with h.
    g.ndata['h'] = features
    g.update_all(msg, reduce)
    g.apply_nodes(func = self.apply_mod)
    return g.ndata.pop('h')

In [30]:
class Classifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_classes):
        super(Classifier, self).__init__()
        self.layers = nn.ModuleList([GCN(in_dim, hidden_dim, F.relu), GCN(hidden_dim, hidden_dim, F.relu)])
        self.classify = nn.Linear(hidden_dim, n_classes)

    def forward(self, g):
        h = g.in_degrees().view(-1, 1).float() #this is basically its features
        for conv in self.layers: 
            h = conv(g, h)
        g.ndata['h'] = h

        hg = dgl.mean_nodes(g,'h')

        
        return self.classify(hg)

In [27]:
'''
the train data and test data loader here

'''
# trainset = MiniGCDataset(320, 10, 20)
# testset = MiniGCDataset(80, 10, 20)
# data_loader = DataLoader(trainset, batch_size = 32, shuffle = True, collate_fn = collate)

'\nthe train data and test data loader here\n\n'

In [29]:
model = Classifier(1, 256, trainset.num_classes) #why???
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
model.train()
losses = []
for epoch in range(80):
    epoch_loss = 0
    for iter, (bg, label) in enumerate(data_loader):
        pred = model(bg)
        loss = criterion(pred, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.detach().item()
    epoch_loss /= (iter + 1)
    print(f"epoch {epoch}| loss{epoch_loss}")
    losses.append(epoch_loss)

NameError: name 'Classifier' is not defined